In [ ]:
from langchain_community.document_loaders import YoutubeLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA

In [ ]:
loader=YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=J5_-l7WIO_w",
    language=["en", "hi"],
    translation="en"
)

transcript=loader.load()
# print(transcript)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # Optimal for OpenAI/GPT use (adjustable)
    chunk_overlap=200,    # Gives LLM some context from previous chunk
    separators=["\n\n", "\n", ".", "!", "?", ",", " "]  # Falls back gradually
)

# print(transcript[0].page_content)
docs=text_splitter.split_documents(transcript)

print(len(docs))
print(docs[0])

Hi guys, my name is Nitesh and you are welcome to my YouTube channel.  In this video also we will continue our lang chain playlist. Uh in the last video, we started studying rag and we focused on discussing the theory around rag.  Did I tell you what rags are?  Why is it needed ?  I also demonstrated comparing racks there using techniques like fine tuning.  And today's video is a continuation of the previous video. Where we will practically create a rag based system using lang chain.  The plan is that I will take a problem statement and create a rig based system around that problem statement and we are going to do all this code in lang chain. So whatever you have read till now in the previous four videos, document loaders, text splitters, vector strings, we will use all these in today's video and using these, we will create a rag base system.  On the whole it's going to be a very interesting video.  Let's start.  So come on guys, first let's talk about the problem statement.  What are 

In [7]:
vectorstore=FAISS.from_documents(
    docs,
    embedding=OpenAIEmbeddings()
)

In [31]:
retriever=vectorstore.as_retriever(search_type="mmr", #this enables MMR
    search_kwargs={'k':3, "lambda_mult":0.5})

In [32]:
prompt=PromptTemplate(
    template="""
    You are a helpful assistant. Answer the following questions only from the provided context. If the context is not sufficient, just say "Sorry don't have enough context to answer this question".
    {context}
    Question: {question}
    """,
    input_variables=['context', 'question']
)

In [ ]:
llm=ChatOpenAI()
question="what are the possible improvements suggested in the chatbot"

In [34]:
rag_chain=(
    retriever
    | (lambda docs:{'context':"\n\n".join([doc.page_content for doc in docs]), 'question':question} )
    | prompt
    | llm
)

In [40]:
question="what are the possible improvements suggested in the chatbot"
response=rag_chain.invoke(question)

print(response.content)

Some possible improvements suggested in the chatbot are:
1. Optimizing the context window to avoid crossing limitations.
2. Implementing citation in the generation step to provide source information for answers.
3. Using the Langmith Bol library for training.
4. Evaluating the entire pipeline using tracers to ensure proper functioning.
5. Improving the indexing process for fetching, splitting, embedding, and storing documents.
6. Fixing errors in auto-generated transcripts for more accurate information.


In [ ]:
"""
Alternate way to create this chain is
1. create a parallel chain that gives the documents fromt the retriever passed to a preprocessing function (lambdafunctions) to concatenate the docs and the other side gives the question as it is. This becomes a dictionary of context and question
2. pass this to prompt | llm | parser
"""